# Train Some Example Models

## Imports


In [1]:
import os
import sys
os.getcwd()
sys.path.append('../preprocess')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
from torchsummary import summary

# Dataset
# from preprocess.GeoGuessrDataset import GeoGuessrDataset
from GeoGuessrDataset import GeoGuessrDataset, ToTensor


import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tnrange

%matplotlib inline

# Enable autoreloading of imported modules.
%load_ext autoreload
%autoreload 2

In [3]:
# Check GPU support on your machine.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

root_dir = r"C:\Users\Shadow\Pictures\Geogussr\Projekt"
HEIGHT = 512
WIDTH = 2560

cuda:0


In [4]:
torch.cuda.get_device_name(device=None)

'NVIDIA RTX A4500'

In [5]:
import torch
import torch.nn as nn

class GeoGuessrNet(nn.Module):
    def __init__(self):
        super(GeoGuessrNet, self).__init__()
        
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=16, stride=6, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=16, stride=6, padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=8, stride=6, padding=1)
        
        # Define the fully-connected layers
        self.fc1 = nn.Linear(704, 200)  # modified to have the correct input size
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(10, 32768)
        # self.fc2 = nn.Linear(1024, 32768)  # output size is still 32768
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.fc2(x) 
        x = self.fc3(x)
        return x


In [6]:
def capacity(module):
    """
    Computes the number of learnable parameters.

    Parameters:
        - module (nn.Module): Module with parameters.

    Returns:
        - num_param (int): Number of parameters.

    """
    num_param = sum(p.numel() for p in module.parameters() if p.requires_grad)

    return num_param

In [7]:
from networks import ResNet
from networks import ResNet2
from networks import TraversedNet
#model_1 = GeoGuessrNet()
#model_2 = ResNet()
#model_3 = ResNet2()
model_4 = TraversedNet()
model = model_4.to(device)
#summary(model, (3,HEIGHT, WIDTH))
print(capacity(model))

47483024


### Define Model and Summary

### Define Optimization

In [8]:
# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
# Define the data transformation
transform = transforms.Compose([
    transforms.ToTensor(),  # convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # normalize images
])

### Load Dataset

In [9]:
# Load the dataset and split it into training and validation sets
# You will need to implement the `GeoLocDataset` class and the `__getitem__` and `__len__` methods

dataset = GeoGuessrDataset(csv_file='../preprocess/coordinates_geohash.csv',
                                    root_dir=root_dir, transform=transform)
train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Define the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=7,pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=7,pin_memory=True)


In [10]:
for i, batch in enumerate(train_dataloader):
    x, y = batch["image"], batch["geohash"]
    print(x.shape, y.shape)
    break

torch.Size([8, 3, 512, 2560]) torch.Size([8, 32768])


### Train Model

In [11]:
import multiprocessing
multiprocessing.cpu_count()

8

In [12]:
torch.backends.cudnn.benchmark = True

scaler = torch.cuda.amp.GradScaler()
accum_iter = 4
# Train the model
num_epochs = 1000
best_loss= 10

batch = next(iter(train_dataloader))

torch.backends.cudnn.benchmark = True
for epoch in (pbar := tnrange(num_epochs)):
    # Training
    model.train()
    train_loss = 0.0
    #for idx, batch in enumerate(train_dataloader):
        
    # Convert images (input) to float because it is a ByteTensor. Why is it a ByteTensor?
    images, labels = batch["image"].float(), batch["geohash"].float()
    #if idx % 1000 == 0:
     #   print(idx)
    # Move the data to the device
    images = images.to(device,non_blocking=True)
    labels = labels.to(device,non_blocking=True)


    with torch.autocast(device_type='cuda', dtype=torch.float16):

        # Forward pass
        outputs = model(images)
        assert outputs.dtype is torch.float16
        loss = criterion(outputs, labels)
        assert loss.dtype is torch.float32

    # Backward pass
    scaler.scale(loss).backward()
    #increase effective batch size and have more stable gradients
    #if ((idx + 1) % accum_iter == 0) or (idx + 1 == len(train_dataloader)):
        # Update the weights
    scaler.step(optimizer)
    scaler.update()
    # Zero the gradients
    optimizer.zero_grad()
    # Accumulate the training loss
    train_loss += loss.item() 
    train_loss = train_loss / 1
        
        #print(train_loss, best_loss)
    #if train_loss < best_loss:
     #   best_loss = train_loss
      #  PATH3 = r"C:\Users\Shadow\Documents\DLCV_Project_GeoGuessr_AI-Basti\models\TraversedNet_best.tar"
       # torch.save({
        #'epoch': epoch,
        #'model_state_dict': model.state_dict(),
        #'optimizer_state_dict': optimizer.state_dict(),
       # 'loss': loss
       # }, PATH3)

    scheduler.step()
    
    pbar.set_description(f'Epoch: {epoch + 1}  Loss: {train_loss:.5f}')
    
        

  0%|          | 0/1000 [00:00<?, ?it/s]

### Saving (change name)

In [13]:
#PATH1 = r"C:\Users\Shadow\Documents\DLCV_Project_GeoGuessr_AI-Basti\models\TraversedNetmodel.pt"
#PATH2 = r"C:\Users\Shadow\Documents\DLCV_Project_GeoGuessr_AI-Basti\models\TraversedNetoptim.pt"
#torch.save(model.state_dict(), PATH1)
#torch.save(optimizer.state_dict(), PATH2)

In [14]:
#PATH3 = r"C:\Users\Shadow\Documents\DLCV_Project_GeoGuessr_AI-Basti\models\TraversedNet_best.tar"
#torch.save({
 #           'epoch': epoch,
  #          'model_state_dict': model.state_dict(),
   #         'optimizer_state_dict': optimizer.state_dict(),
    #        'loss': loss
     #       }, PATH3)

### Load Model State (change Name)

In [15]:



checkpoint = torch.load(r"C:\Users\Shadow\Documents\DLCV_Project_GeoGuessr_AI-Basti\models\TraversedNet_best.tar")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

### Validation

In [16]:
import pygeohash
def decimal_to_geohash(decimal):
    base_32 = '0123456789bcdefghjkmnpqrstuvwxyz'
    geohash = ''
    while decimal > 0:
        geohash += base_32[decimal % 32]
        decimal //= 32
    return geohash[::-1]

def geohash_to_lat_lon(geohash):
    return pygeohash.decode(geohash)


In [17]:
from haversine import haversine

###########################
geogussrscore = []
###########################
# Validation
model.eval()
val_loss = 0.0
total = 0
correct = 0
with torch.no_grad():
    for idx, batch  in enumerate(val_dataloader):
        images, labels = batch["image"].float(), batch["geohash"].float()
        # Move the data to the device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        prediction = torch.argmax(outputs.data, 1)
        labels = torch.argmax(labels.data, 1)
        
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
        
        # Accumulate the validation loss                                
        val_loss += loss.item() * images.size(0)                        
        
        
        #################distance
        distance = []
        
        for i in range(len(images)):
            distance.append(haversine(geohash_to_lat_lon(decimal_to_geohash(prediction.data[i])),geohash_to_lat_lon(decimal_to_geohash(labels.data[i]))))
            #geohashes_pred.append(geohash_to_lat_lon(decimal_to_geohash(prediction.data[i])))
            #geohashes_label.append(geohash_to_lat_lon(decimal_to_geohash(labels.data[i])))
        
        
        #distances = haversine(geohashes_pred, geohashes_label)
        geogussrscore.append((5000*np.exp(-np.array(distance)/2000)).mean())
        
        
        ##################################
    #evaluate accuracy         
accuracy = 100* correct/total

# Print the epoch loss                                                  
#train_loss = train_loss / len(train_dataloader.dataset)                 
val_loss = val_loss / len(val_dataloader.dataset)
average_score = np.array(geogussrscore).mean()
print(average_score)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Shadow\.conda\envs\pytorch\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\Shadow\.conda\envs\pytorch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Shadow\.conda\envs\pytorch\lib\site-packages\torch\utils\data\_utils\fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\Shadow\.conda\envs\pytorch\lib\site-packages\torch\utils\data\dataset.py", line 290, in __getitem__
    return self.dataset[self.indices[idx]]
  File "C:\Users\Shadow\Documents\DLCV_Project_GeoGuessr_AI-valdrin\model\GeoGuessrDataset.py", line 48, in __getitem__
    sample["image"] = self.transform(sample["image"])
  File "C:\Users\Shadow\.conda\envs

In [ ]:
print(accuracy)

In [ ]:
5000*np.exp(-10/2000)

In [ ]:
"""
for i in range(10):
    sample = dataset[i]
    plt.imshow(sample["image"].permute(1, 2, 0))
    print(i, sample['image'].shape, sample['geohash'])
"""